In [1]:
from fonctions import *

In [17]:
# chargement des étiquettes
dic_tags_stanford = lire_json("articlesTags_stanford.json")

new_dic = reorganiser_POS(dic_tags_stanford)

In [18]:
# séparation des informations dans trois listes différentes

chemins_articles = [x for x in new_dic.keys()]

liste_tags = [x[1] for x in new_dic.values()]

liste_titres = [x[2] for x in new_dic.values()]

In [9]:
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

X, V = creer_X(liste_tags, 4, 4)

# chargements des entités nommées
dic_EN = lire_json('entites_nommees.json')
liste_tagEN = [x[1] for x in dic_EN.values()]
liste_titres = [x[2] for x in dic_EN.values()]

X_en, V_en = creer_X(liste_tagEN, 1, 1)

# transformation en sparse matrix
sparse = csr_matrix(X_en)

# fusion des deux vectorisation
X_fusion = hstack((X, sparse))

[0. 0. 0. 0.]
(5685, 4)


In [13]:
model = creer_model_KM(8, X_fusion)

predictions = model.predict(X_fusion)

print(predictions[:100])
print(effectifs_clusters(model))

[2 1 2 2 0 1 4 0 3 0 4 3 2 0 2 4 3 2 0 2 0 0 4 2 3 2 4 3 3 3 3 3 1 4 3 3 1
 1 1 1 3 1 4 4 3 3 3 4 3 3 1 1 0 1 1 2 0 0 4 3 1 1 1 3 2 1 2 3 1 2 3 3 2 3
 2 3 1 1 1 0 4 3 4 1 3 2 4 3 3 4 3 4 3 3 0 2 4 0 4 4]
{2: 1390, 1: 1050, 0: 764, 4: 868, 3: 1613}


In [14]:
# redéfinition des fonctions existantes pour gérer l'affichage 
# à la fois des entités nommées et des tags de chaque titre

def dic_complet2(X, predictions, liste_tags, liste_titres, liste_tagsEN):
    dic_res = {}
    for i in range(len(X)):
        num_cluster = predictions[i]
        titre = liste_titres[i]
        tags = liste_tags[i]
        en = liste_tagsEN[i]
        vecteur = X[i]
        dic_res.setdefault(num_cluster,  [])
        dic_res[num_cluster].append((titre, tags, en, vecteur))
    return dic_res

def afficher_titres_hasard2(taille_echantillons, dictionnaire):
    echantillons = []
    tags = []
    tagsEN = []
    for i in range(len(dictionnaire)):
        echantillons.append([])
        tags.append([])
        tagsEN.append([])
        for j in range(taille_echantillons):
            indice = randint(0, len(dictionnaire[i])-1)
            echantillons[i].append(dictionnaire[i][indice][0])
            tags[i].append(dictionnaire[i][indice][1])
            tagsEN[i].append(dictionnaire[i][indice][2])

    for i in range(len(echantillons)):
        print("Cluster %s : %s articles" % (i, len(dictionnaire[i])))
        for j in range(len(echantillons[i])):
            print(echantillons[i][j])
            print(" ".join(tags[i][j]))
            print(" ".join(tagsEN[i][j]))
        print("-"*10)

In [15]:
# extraction de titres d'articles au hasard pour chaque cluster

dictionnaire = dic_complet2(X, predictions, liste_tags, liste_titres, liste_tagEN)

afficher_titres_hasard2(2, dictionnaire)

Cluster 0 : 764 articles
Dr Komla Siamévi, représentant de l'Oms :'Ce sera dramatique si les Arv perdent leur efficacité''
PROPN PROPN PROPN PUNCT NOUN ADP NOUN PUNCT NOUN AUX ADJ SCONJ DET PROPN VERB DET NOUN PUNCT
MISC ORG LOC
CULTURE DE CONTRE SAISON : 122 groupements de Tambacounda soutenus par la Fao
NOUN ADP PROPN PROPN PUNCT NUM NOUN ADP PROPN VERB ADP DET PROPN
MISC LOC ORG
----------
Cluster 1 : 1050 articles
Robert Mugabe sommé de quitter le pouvoir
PROPN PROPN VERB ADP VERB DET NOUN
PER
Mauritanie / Santé Démarrage de la 3ème phase de la campagne nationale de vaccination contre le tétanos Nouakchott, 14 déc (AMI) - Le ministre de la santé, M. Mohamed Abdallahi Ould Siyam a supervisé dimanche au dispensaire de la Socogim dans la moughataa du Ksar le démarrage de la 3ème phase de la....
PROPN PUNCT NOUN NOUN ADP DET ADJ NOUN ADP DET NOUN ADJ ADP NOUN ADP DET NOUN PROPN PUNCT NUM NOUN PUNCT NOUN PUNCT PUNCT DET NOUN ADP DET NOUN PUNCT NOUN PROPN PROPN PROPN PROPN AUX VERB NOUN 